In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os, random
import torch
from transformers import BertTokenizer, BertForSequenceClassification

from CAP import CAPModel
from CAP.dataset import CAPDataSet
from CAP.utils import *

seed_everything(1813)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Functions

In [2]:
def preprocess_data(dataframe, tokenizer, max_len=64):
    input_ids = []
    attention_masks = []
    labels = []

    for _, row in dataframe.iterrows():
        encoded = tokenizer.encode_plus(
            str(row['texts'].upper()),  # 문자열로 변환하여 전달
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True  # truncation 활성화
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        labels.append(int(row['labels']))  # 정수형으로 변환하여 추가

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)  # 정수형 리스트를 텐서로 변환

    return TensorDataset(input_ids, attention_masks, labels)

## Define model

In [3]:
model = BertForSequenceClassification.from_pretrained('bert-large-cased', num_labels=5).to(device)

C:\Users\GangsuKim\Anaconda3\envs\cap_test\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

C:\Users\GangsuKim\Anaconda3\envs\cap_test\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\GangsuKim\.cache\huggingface\hub\models--bert-large-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
